In [ ]:
# %pip install -U pygis lidar geedim

In [ ]:
import os
import ee
import geemap
import lidar
import pandas as pd

In [ ]:
m = geemap.Map(center=[47.2121, -99.0280], zoom=9, height=800)

url = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}"
m.add_tile_layer(url, name="Google Satellite", attribution="Google")

dem_wms = "https://elevation.nationalmap.gov/arcgis/services/3DEPElevation/ImageServer/WMSServer"
layer = "3DEPElevation:Hillshade Multidirectional"
m.add_wms_layer(
    url=dem_wms, layers=layer, name="Hillshade", format="image/png", shown=False
)


huc12 = ee.FeatureCollection("USGS/WBD/2017/HUC12")
huc08 = ee.FeatureCollection("USGS/WBD/2017/HUC08")
collection = ee.ImageCollection("USGS/3DEP/1m")
style = {"color": "0000ff88", "fillColor": "00000000", "width": 1}
m.add_layer(
    collection, {"min": 0, "max": 4000, "palette": "terrain"}, "3DEP", False, 0.5
)

nwi_wms = "https://fwspublicservices.wim.usgs.gov/wetlandsmapservice/services/Wetlands/MapServer/WMSServer"
m.add_wms_layer(url=nwi_wms, layers="1", name="NWI", format="image/png", shown=False)

m.add_layer(huc12, {}, "HU-12 Vector", False)
m.add_layer(
    huc12.style(**style),
    {},
    "HU-12",
)
m.add_layer(huc08, {}, "HU-8 Vector", True, 0.5)

m.add_layer_manager(opened=False)
m

Pipestem HU8: 10160002, 10130103
Pipestem HU12: 101600020502, 101301030306

In [ ]:
if m.user_roi is not None:
    huc12_id = huc12.filterBounds(m.user_roi).first().get("huc12").getInfo()
else:
    # huc12_id = '101303050505'
    huc12_id = "101301030306"

print(huc12_id)

In [ ]:
roi = huc12.filter(ee.Filter.eq("huc12", huc12_id))
m.addLayer(
    roi.style(**{"color": "ff0088", "fillColor": "00000000", "width": 2}), {}, "ROI"
)
images = collection.filterBounds(roi)
size = images.size().getInfo()
print(f"Number of images: {size}")

In [ ]:
if size > 0:
    image = images.median().clipToCollection(roi).setDefaultProjection("EPSG:5070")
    hillshade = ee.Terrain.hillshade(image)
    window_size = 2
    reducer = ee.Reducer.mean()
    kernel = ee.Kernel.square(radius=window_size, units="pixels")
    dem = image.reduceNeighborhood(**{"reducer": reducer, "kernel": kernel})
    dem_hs = ee.Terrain.hillshade(dem)

    m.add_layer(image, {"min": 1000, "max": 3000, "palette": "terrain"}, "DEM")
    m.add_layer(hillshade, {}, "Hillshade")
    m.add_layer(dem_hs, {}, "Hillshade_smoothed")
else:
    print("No data available for the selected HUC12")

In [ ]:
if size > 0:
    if not os.path.exists(f"{huc12_id}.tif"):
        geemap.download_ee_image(
            dem, f"{huc12_id}.tif", scale=3, crs="EPSG:5070", region=roi.geometry()
        )
    else:
        print(f"{huc12_id}.tif already exists")

In [ ]:
lidar.ExtractSinks(f"{huc12_id}.tif", min_size=100, out_dir=huc12_id)

In [ ]:
m = geemap.Map(center=[40, -100], zoom=5)
url = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}"
m.add_tile_layer(url, name="Google Satellite", attribution="Google")
m.add_layer(dem_hs, {}, "Hillshade_smoothed")
m.add_layer(image, {"min": 0, "max": 3000, "palette": "terrain"}, "DEM", False)
# m.add_raster(f"{huc12_id}.tif", colormap="terrain", layer_name="DEM")

hu8_dep_id = "users/giswqs/depressions/" + huc12_id[:8]
hu8_dep = ee.FeatureCollection(hu8_dep_id).filterBounds(roi)
m.add_layer(hu8_dep, {}, "HU8 depressions", False)

style = {"color": "#0000ff", "fillColor": "#0000ff"}
m.add_vector(
    f"{huc12_id}/{huc12_id}.gpkg",
    style=style,
    layer_name="HU12 depressions",
)
m.addLayer(
    roi.style(**{"color": "ff0088", "fillColor": "00000000", "width": 2}), {}, "ROI"
)

nwi_wms = "https://fwspublicservices.wim.usgs.gov/wetlandsmapservice/services/Wetlands/MapServer/WMSServer"
m.add_wms_layer(url=nwi_wms, layers="1", name="NWI", format="image/png")

nhd_flowline = ee.FeatureCollection(
    "projects/sat-io/open-datasets/NHD/NHD_ND/NHDFlowline"
)
m.add_layer(nhd_flowline, {}, "NHD Flowline", False)

wetlands = ee.FeatureCollection(
    "projects/sat-io/open-datasets/NWI/wetlands/ND_Wetlands"
)
m.add_layer(wetlands, {}, "Wetlands", False)

m.add_layer_manager(opened=False)
m.centerObject(roi)
m

In [ ]:
huc12_id = "101303050505"

In [ ]:
# lidar.download_3dep_1m(huc12_id, '3m.tif', scale=3)

In [ ]:
# lidar.download_3dep_10m(huc12_id, '10m.tif', scale=10)

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/hydrology/huc12.csv"
df = pd.read_csv(url, dtype=str)
df

In [ ]:
hu8_id = "10130103"

In [ ]:
hu12_ids = df[df["huc12"].str.startswith(hu8_id)]
hu12_ids = hu12_ids["huc12"].tolist()
print(f"Number of HUC12s: {len(hu12_ids)}")

In [ ]:
work_dir = os.path.join(os.path.expanduser("~"), "Downloads")

In [ ]:
for huc12_id in hu12_ids:
    print(huc12_id)
    out_dir = os.path.join(work_dir, huc12_id[:8])
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    filename = f"{out_dir}/{huc12_id}.tif"
    lidar.download_3dep_1m(huc12_id, filename, scale=3)

In [ ]:
filename = os.path.join(work_dir, f"{hu8_id}.tif")
lidar.download_3dep_1m(hu8_id, filename, scale=3)

In [ ]:
out_dir = os.path.join(work_dir, hu8_id)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
lidar.ExtractSinks(filename, min_size=100, out_dir=out_dir)